In [4]:
from ase.io import read, write
from ase.visualize import view

materials = ['TiN', 'VN', 'ScN','NbN','ZrN']
crystal_dir = './Crystals'
for material in materials:
    crystal = read(crystal_dir+f'/{material}_ase_opt.traj')
    view(crystal)


In [8]:
from ase.io import read, write
from ase.atoms import Atoms
import os

materials = ['TiN', 'VN', 'ScN', 'NbN', 'ZrN']
crystal_dir = './Crystals'
fixed_index_dir = crystal_dir+'/Fixed_Index_Crystals' # Directory to save reordered structures

# Create the directory if it doesn't exist
os.makedirs(fixed_index_dir, exist_ok=True)

for material in materials:
    print(f"Processing {material} to fix atom indexing...")
    crystal_path = os.path.join(crystal_dir, f'{material}_ase_opt.traj')
    
    if not os.path.exists(crystal_path):
        print(f"Warning: {crystal_path} not found. Skipping {material}.")
        continue

    crystal = read(crystal_path)

    # Get a list of atoms with their chemical symbols and original indices
    # We will sort this list to create a new Atoms object with desired order
    atom_list = []
    for atom in crystal:
        atom_list.append((atom.symbol, atom.position, atom.tag, atom.momentum, atom.mass, atom.magmom))
    
    # Sort atoms first by chemical symbol (e.g., 'N' before 'Nb', 'Ti' before 'N')
    # This assumes consistent naming for metal and nitrogen atoms.
    # For example, if 'M' is the metal and 'N' is nitrogen, 'M' will come before 'N' due to alphabetical sort.
    atom_list.sort(key=lambda x: x[0]) 

    # Create a new Atoms object with the reordered atoms
    # Extract positions, symbols, and cell/pbc from original crystal
    new_symbols = [item[0] for item in atom_list]
    new_positions = [item[1] for item in atom_list]
    
    reordered_crystal = Atoms(symbols=new_symbols,
                              positions=new_positions,
                              cell=crystal.get_cell(),
                              pbc=crystal.get_pbc())

    # Add tags, momentum, mass, magmom if they were present
    if crystal.has('tags'):
        reordered_crystal.set_tags([item[2] for item in atom_list])
    if crystal.has('momenta'):
        reordered_crystal.set_momenta([item[3] for item in atom_list])
    if crystal.has('masses'):
        reordered_crystal.set_masses([item[4] for item in atom_list])
    if crystal.has('magmoms'):
        reordered_crystal.set_magmoms([item[5] for item in atom_list])

    # Save the reordered crystal to an .xyz file
    xyz_filename = os.path.join(fixed_index_dir, f'{material}_reordered.xyz')
    write(xyz_filename, reordered_crystal)
    
    print(f"Reordered atoms and saved to {xyz_filename}")
    print(f"New atom order for {material}: {reordered_crystal.get_chemical_formula()} (original number of atoms: {len(crystal)})")

print("\nAtom indexing fix process completed for all materials.")

Processing TiN to fix atom indexing...
Reordered atoms and saved to ./Crystals/Fixed_Index_Crystals\TiN_reordered.xyz
New atom order for TiN: N4Ti4 (original number of atoms: 8)
Processing VN to fix atom indexing...
Reordered atoms and saved to ./Crystals/Fixed_Index_Crystals\VN_reordered.xyz
New atom order for VN: N4V4 (original number of atoms: 8)
Processing ScN to fix atom indexing...
Reordered atoms and saved to ./Crystals/Fixed_Index_Crystals\ScN_reordered.xyz
New atom order for ScN: N4Sc4 (original number of atoms: 8)
Processing NbN to fix atom indexing...
Reordered atoms and saved to ./Crystals/Fixed_Index_Crystals\NbN_reordered.xyz
New atom order for NbN: N4Nb4 (original number of atoms: 8)
Processing ZrN to fix atom indexing...
Reordered atoms and saved to ./Crystals/Fixed_Index_Crystals\ZrN_reordered.xyz
New atom order for ZrN: N4Zr4 (original number of atoms: 8)

Atom indexing fix process completed for all materials.


In [9]:
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
from ase.constraints import FixAtoms

materials = ['TiN', 'VN', 'ScN', 'NbN', 'ZrN']
crystal_dir = './Crystals/Fixed_Index_Crystals' # Directory with fixed index crystals
slab_dir = './Slabs' # Directory to save slab structures

# Create the Slabs directory if it doesn't exist
import os
os.makedirs(slab_dir, exist_ok=True)

for material in materials:
    print(f"Processing {material}...")
    crystal = read(crystal_dir + f'/{material}_reordered.xyz')

    # Define Miller index (e.g., (100))
    miller_index = (1, 0, 0)
    
    # Create the slab structure
    # vacuum: thickness of the vacuum layer in Angstroms
    # layers: total number of layers in the slab
    slab = surface(crystal, miller_index, layers=2, vacuum=10.0)

    # Create a 2x2 supercell in the x-y plane
    # The '1' in the third dimension ensures no repetition along the z-axis (perpendicular to surface)
    slab = slab * (2, 2, 1)

    # Apply constraints: Fix the bottom 2 layers
    # Determine the number of atoms in the fixed layers
    # Assuming layers are ordered from bottom to top by the surface function
    # You might need to adjust this based on how 'surface' orders atoms,
    # but generally, the first 'n' layers correspond to the bottom.
    
    # Get z-coordinates to identify layers
    z_coords = slab.get_positions()[:, 2]
    unique_z = sorted(list(set(z_coords)))

    # Identify the z-coordinates of the bottom two layers
    fixed_z_threshold = unique_z[1] # The second unique z-coordinate from the bottom

    # Create a list of indices of atoms to be fixed
    fixed_indices = [atom.index for atom in slab if atom.position[2] <= fixed_z_threshold]
    
    # Apply the FixAtoms constraint
    slab.set_constraint(FixAtoms(indices=fixed_indices))

    print(f"Slab for {material} created with {slab.get_global_number_of_atoms()} atoms.")
    print(f"Number of fixed atoms: {len(fixed_indices)}")
    
    # View the slab (optional, useful for verification)
    view(slab)

    # Write the slab structure to a file
    write(slab_dir + f'/{material}_slab_{miller_index[0]}{miller_index[1]}{miller_index[2]}.traj', slab)
    print(f"Slab saved to {slab_dir}/{material}_slab_{miller_index[0]}{miller_index[1]}{miller_index[2]}.traj")

print("\nAll slab structures have been created and saved.")

Processing TiN...
Slab for TiN created with 64 atoms.
Number of fixed atoms: 32
Slab saved to ./Slabs/TiN_slab_100.traj
Processing VN...
Slab for VN created with 64 atoms.
Number of fixed atoms: 32
Slab saved to ./Slabs/VN_slab_100.traj
Processing ScN...
Slab for ScN created with 64 atoms.
Number of fixed atoms: 32
Slab saved to ./Slabs/ScN_slab_100.traj
Processing NbN...
Slab for NbN created with 64 atoms.
Number of fixed atoms: 32
Slab saved to ./Slabs/NbN_slab_100.traj
Processing ZrN...
Slab for ZrN created with 64 atoms.
Number of fixed atoms: 32
Slab saved to ./Slabs/ZrN_slab_100.traj

All slab structures have been created and saved.
